In [1]:
!pip install --quiet --upgrade openai tenacity pandas ipywidgets
!pip install tiktoken
!pip install sentence_transformers scikit-learn



In [2]:
import os, time, json, math
import pandas as pd
import ipywidgets as widgets
import llm_utils

from IPython.display import display, Markdown,FileLink

from llm_utils import chat_completion,OutlineStep, parse_llm_output,DetailStep
from typing import List, Dict,Any
from validation import MATERIAL_DATA  
from validation import validate_plan, add_power_check   
df_full_valid: pd.DataFrame | None = None
from embed_utils import fetch_examples





In [3]:
# === 2. 定义 UI 元素 ===
desc_input = widgets.Textarea(
    value="An aluminum gear with 20 teeth and a central bore.",
    placeholder="Describe the part here...",
    description="Part:",
    layout=widgets.Layout(width="100%", height="100px")
)

material_selector = widgets.Dropdown(
    options=['aluminum', 'steel', 'brass', 'titanium', 'plastic'],
    value='aluminum',
    description='Material:',
    layout=widgets.Layout(width='30%')
)


generate_button = widgets.Button(
    description=" Generate CNC Plan",
    button_style="success",
    layout=widgets.Layout(width="30%", margin="10px 0")
)

# === Export Button ===
export_btn = widgets.Button(
    description="Export CSV",
    icon="download",
    button_style='',                       # neutral gray
    layout=widgets.Layout(width='30%', margin='5px 0 15px 0')
)

def on_export_clicked(b):
    global df_full_valid
    with output_area:                     # ★ 关键：写进同一输出框
        if df_full_valid is None:
            print("Please generate a plan first.")
            return

        fname = f"plan_{material_selector.value}_{pd.Timestamp.today().date()}.csv"
        df_full_valid.to_csv(fname, index=False)
        print(f"Saved as {fname}")

    
        display(FileLink(fname, result_html_prefix="Download: "))

export_btn.on_click(on_export_clicked)
output_area = widgets.Output()

In [4]:
def get_outline(part: str, material: str, max_retries: int = 3) -> List[Dict[str, str]]:
    """
    Ask the LLM for a high-level process outline and
    return a list of {"step": "...", "description": "..."} dictionaries.
    """
    system_msg = {
        "role": "system",
        "content": (
            "You are a CNC process planner. "
            "Return ONLY a JSON array."
            "Each item must have keys 'step' and 'description'."
        )
    }
    user_msg = {
        "role": "user",
        "content": (
            f"The part is: {part}\n"
            f"The material is: {material}\n"
            "List the high-level manufacturing steps needed to machine this part."
            "For each step, include:\n"
            "- step: the step name\n"
            "- description: a short description of what happens in this step\n\n"
            "Return ONLY a JSON array. No explanations, no markdown."
        )
    }

    for attempt in range(1, max_retries + 1):
        try:
            raw = chat_completion(messages=[system_msg, user_msg], verbose=False)
            return parse_llm_output(raw, OutlineStep)
        except Exception as e:
            print(f"get_outline attempt {attempt} parsing failed: {e}")
            if attempt == max_retries:
                return []
            print("Retrying get_outline…")
    
    
def get_detail(outline: List[Dict[str, str]] | str,
               part: str,
               material: str,
               max_retries: int = 3) -> List[Dict[str, Any]]:
    """
    Enrich each step with tool, operation, rpm and feed.
    Returns raw JSON string.
    """

    # --- 1. Convert outline list to bullet list text -----------------
    outline_text = (
        "\n".join(f"- {s['step']}" for s in outline if 'step' in s)
        if isinstance(outline, list) else outline
    )

    # --- 2. Fetch material-specific limits --------------------------
    limits =MATERIAL_DATA[material]
    rpm_min, rpm_max = limits["rpm"]
    feed_min, feed_max = limits["feed"]
    if limits is None:
        # fallback to a safe generic range
        limits = {"rpm": (500, 5000), "feed": (100, 1000)}

    material_constraints = (
    f"For **{material}**, spindle speed **must be {rpm_min}–{rpm_max} rpm**, "
    f"and feed rate **must be {feed_min}–{feed_max} mm/min**. "
    "Stay strictly within these ranges."
    )

    
    # === 插入向量检索示例 ===
    few_shot = fetch_examples(part, material, k=2)
    combined_examples = []
    for ex in few_shot:
        combined_examples.extend(ex)
        
    example_block = json.dumps(combined_examples, ensure_ascii=False)
        
    # --- 3. Build messages list for chat_completion -----------------
    system_msg = {
        "role": "system",
        "content": "You are a CNC process planner."
    }

    user_msg = {
        "role": "user",
        "content": (
            f"The part is: {part}\n"
            f"The material is: {material}\n"
            "Here are similar part examples (JSON):\n"
            f"{example_block}\n"
            f"Here is the outline of steps:\n{outline_text}\n\n"
            "For EACH step output an object with: step, tool, operation, rpm, feed.\n"
            f"{material_constraints}\n"
            "For non-machining steps (e.g. setup, inspection) set rpm=0 and feed=0.\n\n"
            "Each item must have keys 'step', 'tool', 'operation', 'rpm', 'feed'.\n\n"
            "Return ONLY a JSON array. No explanations. No markdown."
        )
    }

    for attempt in range(1, max_retries + 1):
        try:
            raw = chat_completion(messages=[system_msg, user_msg], verbose=False)
            return parse_llm_output(raw, DetailStep)
        except Exception as e:
            print(f"get_detail attempt {attempt} parsing failed: {e}")
            if attempt == max_retries:
                return []
            print("Retrying get_detail…")


In [5]:
def display_plan_table(df):
    def highlight_invalid(v):
        return "background-color:#FFD2D2" if v is False else ""
    styled = df.style.map(highlight_invalid, subset=["RPM Valid","Feed Valid","Power Valid"])
    
    display(Markdown("### CNC Process Plan"))
    display(styled)


In [6]:
# === 工艺总结 ===
def reflect_summary(raw_json: str, validated_df: pd.DataFrame, material: str):
    num_steps = len(validated_df)
    num_invalid_rpm   = (~validated_df["RPM Valid"]).sum()
    num_invalid_feed  = (~validated_df["Feed Valid"]).sum()
    num_invalid_power = (~validated_df["Power Valid"]).sum()

    comment = (
        "### Reflection Summary\n"
        f"- **Total Steps Generated**: {num_steps}\n"
        f"- **Invalid Spindle Speeds**: {num_invalid_rpm} step(s)\n"
        f"- **Invalid Feed Rates**: {num_invalid_feed} step(s)\n"
        f"- **Invalid Power**        : {num_invalid_power}\n"
        f"- **Power limit for {material}**: "
        f"{MATERIAL_DATA.get(material, {}).get('power', 5.0):.1f} kW\n"
        
    )

    latest = max((p for p in os.listdir() if p.startswith("plan_")), default=None)
    if latest:
        comment += f"- **Exported file**        : `{latest}`\n"

    
    if num_invalid_rpm or num_invalid_feed or num_invalid_power:
        comment += "- ** Human Oversight Needed**:\n"
        if num_invalid_rpm:
            comment += "  - Some spindle speeds out of range.\n"
        if num_invalid_feed:
            comment += "  - Some feed rates out of range.\n"
        if num_invalid_power:
            comment += "  - Some power values exceed machine limit (possible overload).\n"
    else:
        comment += "- All parameters are within expected machining constraints.\n"

    # 统计 token
    comment += f"- **Tokens used so far**: {llm_utils.TOKENS_USED}\n"

    display(Markdown(comment))


In [7]:
# === 4. 按钮点击逻辑 ===
def on_generate_clicked(b): 
    global df_full_valid, df_cut_valid
    with output_area:
        output_area.clear_output()

        part = desc_input.value.strip()
        material = material_selector.value

        if not part:
            print("Please enter the part description.")
            return

        # --- ① 先取高阶大纲 ---------------------------------
        outline = get_outline(part, material)
        if not outline:
            print("Failed to retrieve outline. Please check your network connection or API key.")
            return

        # --- ② 再根据大纲生成带参数的完整 JSON ---------------
        raw_json = get_detail(outline, part, material)

        # --- ③ 解析 LLM JSON → DataFrame -------------------
        df_full = pd.DataFrame(raw_json).reset_index(drop=True)
        df_full.rename(columns={
            "rpm":  "Spindle Speed (RPM)",
            "feed": "Feed Rate (mm/min)"
        }, inplace=True)
        
        if df_full.empty:
            print("JSON parsing failed, raw output:\n", raw_json)
            return

        # --- ④ 全流程校验并展示 ------------------------------
        df_full_valid = validate_plan(df_full, material)
        df_full_valid = add_power_check(df_full_valid, material)
        
        display(Markdown("### Full Process Plan (incl. non-machining)"))
        display_plan_table(df_full_valid)
        
        
        display(Markdown("---"))
        
        # --- ⑤ 过滤仅含切削参数的步骤 -------------------------
        df_cut = df_full[
            df_full["Spindle Speed (RPM)"].gt(0) &
            df_full["Feed Rate (mm/min)"].gt(0)
        ].reset_index(drop=True)

        df_cut_valid = validate_plan(df_cut, material)
        df_cut_valid = add_power_check(df_cut_valid, material)

        display(Markdown("### Machining-only Plan"))
        display_plan_table(df_cut_valid)
        

        # --- ⑥ 反思摘要（基于完整流程） -----------------------
        reflect_summary(raw_json, df_full_valid, material)
       
        

In [8]:

# === 5. 按钮事件绑定 ===
generate_button.on_click(on_generate_clicked)

# === 6. 显示UI（必须放最后） ===
output_area = widgets.Output()
display(
    Markdown("## CNC Process Planner"),
    desc_input,
    material_selector,
    generate_button,
    export_btn, 
    output_area
)  


## CNC Process Planner

Textarea(value='An aluminum gear with 20 teeth and a central bore.', description='Part:', layout=Layout(height…

Dropdown(description='Material:', layout=Layout(width='30%'), options=('aluminum', 'steel', 'brass', 'titanium…

Button(button_style='success', description=' Generate CNC Plan', layout=Layout(margin='10px 0', width='30%'), …

Button(description='Export CSV', icon='download', layout=Layout(margin='5px 0 15px 0', width='30%'), style=But…

Output()